In [5]:
import bs4 as bs
import urllib.request
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [15]:
# Reading the html file
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods').read()

# Finding the table
soup = bs.BeautifulSoup(source, 'html.parser')
tables = soup.findAll('table', attrs={'class':'wikitable sortable'})

data = [] 

for table in tables:

    # Get all table rows
    table_rows = table.find_all('tr')

    # Taking the column names of the table
    ths = table_rows[0].find_all('th')
    cols =[th.text.strip().replace(" ", "") for th in ths]

    # Extracting all the data in the table
    table_rows[0]
    for tr in table_rows:
        tds = tr.find_all('td')
        row = [td.text.strip() for td in tds]
        if len(row) > 0:
            data.append(row)

In [16]:
ncols = ['Neighbourhood','Municipality','Population','LandArea','Density','PopulationChange','AvgIncome','Commuting','Renters','SecondLng']
df_table = pd.DataFrame(data, columns=cols)[(cols[0:2]+cols[3:11])]
df_table.columns = ncols
df_table.head()

,Neighbourhood,Municipality,Population,LandArea,Density,PopulationChange,AvgIncome,Commuting,Renters,SecondLng
0,Crescent Town,EY,"8,157",0.4,"20,393",-10.0,"23,021",24.5,20.3,Bengali (18.1%)
1,Governor's Bridge/Bennington Heights,EY,"2,112",1.87,1129,4.0,"129,904",7.1,13.3,Polish (1.4%)
2,Leaside,EY,"13,876",2.81,4938,3.0,"82,670",9.7,10.5,Bulgarian (0.4%)
3,O'Connor–Parkview,EY,"17,740",4.94,3591,-6.1,"33,517",15.8,19.4,Urdu (3.2%)
4,Old East York,EY,"52,220",7.94,6577,-4.6,"33,172",22.0,19.1,Greek (4.3%)


## Data Cleaning

We can see that raw table have numbers as text and also SecondLng Column needs spliting to the Language and the corresponding percentage. Let's clean the dataset by removing and converting numeric columns.
1. Remove empty lines in the dataset.
2. Remove the summary column at the bottom of the table.
3. Replace Municipality code with the name.
4. Split SecondLng to two columns.
5. Convert all numbers to float values by removing commas in the string.

In [17]:
# Removing empty lines in the dataset and the summary column at the bottom of the table.
df_table = df_table[~(df_table['Municipality']=='')].copy()
# Converting all numbers to float values by removing commas in the string.
df_table[df_table.columns[2:-1]] = df_table[df_table.columns[2:-1]].replace({',':''},regex=True).apply(pd.to_numeric,1)
# Split SecondLng to two columns.
df_table[['SecondLng','SecondLngPct']] = df_table['SecondLng'].str.split('(',expand=True)
df_table['SecondLngPct'] = df_table['SecondLngPct'].replace({'%\)':''},regex=True).apply(pd.to_numeric,1)
# Replacing Municipality code with the name.
df_table['Municipality'] = df_table['Municipality'].replace({'OCoT':'Old City of Toronto',
                          'S':'Scarborough',
                          'NY':'North York',
                          'E':'Etobicoke',
                          'Y':'York',
                          'EY':'East York'})
print('Datatypes of the dataset')
print(str(df_table.dtypes))
df_table.head()
df_table.to_csv("demographics.csv", index=False)

Datatypes of the dataset
Neighbourhood        object
Municipality         object
Population          float64
LandArea            float64
Density             float64
PopulationChange    float64
AvgIncome           float64
Commuting           float64
Renters             float64
SecondLng            object
SecondLngPct        float64
dtype: object


### Reading from the previously saved csv file which has Lat, Lng

In [174]:
demographic_data =  pd.read_csv("demographics.csv")

In [175]:
neighborhood_data = pd.read_csv('latlng.csv')
neighborhood_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Foursqure credentials

### First we visulaize the locations of each neighborhood in Toronto

Lets use folium to show where each of the neighborhood is located in the map.

In [177]:
address = 'City of Toronto, ON'

geolocator = Nominatim(user_agent="TN_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of City of Toronto are 43.6534817, -79.3839347.


In [178]:
# Lets create markers for all the neighbourhoods in Toronto in a Folium map which is centered on Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_data['Latitude'], neighborhood_data['Longitude'], neighborhood_data['Borough'], neighborhood_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Exploring Neighborhoods in Toronto

## 1.1 Getting venue list for each neighbourhood
First lets create a function to get data from Foursqure api.

In [179]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_combined_list=[]
    print('Retriving venues for all the Neighborhoods')
    for name, lat, lng in zip(names, latitudes, longitudes):

            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_combined_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    print('Data collected from Foursqure API')

    nearby_venues = pd.DataFrame([item for venue_list in venues_combined_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [180]:
toronto_venues = getNearbyVenues(neighborhood_data['Neighbourhood'],neighborhood_data['Latitude'],neighborhood_data['Longitude'])

Retriving venues for all the Neighborhoods
Data collected from Foursqure API


In [181]:
toronto_venues.to_csv("venues.csv", index=False)

Let's check how many venues were returned for each neighborhood

In [182]:
toronto_venues[['Neighborhood','Venue','Venue Category']].groupby('Neighborhood').count()

,Venue,Venue Category
Neighborhood,,
Agincourt,4,4
"Alderwood, Long Branch",8,8
"Bathurst Manor, Wilson Heights, Downsview North",20,20
Bayview Village,4,4
"Bedford Park, Lawrence Manor East",23,23
Berczy Park,62,62
"Birch Cliff, Cliffside West",4,4
"Brockton, Parkdale Village, Exhibition Place",100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16


Let's find out how many unique categories can be curated from all the returned venues

In [183]:
print('There are {} uniques categories.'.format(toronto_venues['Venue Category'].nunique()))

There are 285 uniques categories.


# 1.2 Analysing Fast Food Resturent Distribution

In [184]:
# Extracting fast food places
fastfood_places = toronto_venues.query("`Venue Category` == 'Fast Food Restaurant'")

In [185]:
print(f"There are {fastfood_places.shape[0]} fast food resturants in Torronto area")
fastfood_places

There are 32 fast food resturants in Torronto area


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
104,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
150,"Garden District, Ryerson",43.657162,-79.378937,Crepe Delicious,43.654536,-79.380889,Fast Food Restaurant
566,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
603,Thorncliffe Park,43.705369,-79.349372,A&W,43.706275,-79.344670,Fast Food Restaurant
636,"Richmond, Adelaide, King",43.650571,-79.384568,The Burger's Priest,43.648643,-79.387539,Fast Food Restaurant
801,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Bourbon St. Grill,43.778276,-79.343241,Fast Food Restaurant
802,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,KFC,43.777600,-79.344200,Fast Food Restaurant
807,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,McDonald's,43.778407,-79.343574,Fast Food Restaurant
829,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Fit For Life,43.779352,-79.350568,Fast Food Restaurant


In [186]:
# Lets create markers for all the neighbourhoods in Toronto in a Folium map which is centered on Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.2)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_data['Latitude'], neighborhood_data['Longitude'], neighborhood_data['Borough'], neighborhood_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        opacity=0.3,
        parse_html=False).add_to(map_toronto) 
    
for lat, lng, venue, neighborhood in zip(fastfood_places['Neighborhood Latitude'], fastfood_places['Neighborhood Longitude'], fastfood_places['Venue'], fastfood_places['Neighborhood']):
    label = '{}, {}'.format(neighborhood, venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='r',
        fill_opacity=0.9,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

We can see that fast food resturents are mostly located in the city center areas, and there are good places to put new fast food resturents in the city.

In [187]:
#Ranking Neighbohoods with most fastfood resturants
fast_neigh = fastfood_places.groupby('Neighborhood')['Venue'].count().sort_values(ascending=False).to_frame().reset_index()
fast_neigh

,Neighborhood,Venue
0,"Fairview, Henry Farm, Oriole",4
1,Church and Wellesley,2
2,"Steeles West, L'Amoreaux West",2
3,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",2
4,"Willowdale, Willowdale East",1
5,"India Bazaar, The Beaches West",1
6,"Business reply mail Processing Centre, South C...",1
7,"Clarks Corners, Tam O'Shanter, Sullivan",1
8,"Commerce Court, Victoria Hotel",1
9,"First Canadian Place, Underground city",1


Fairview, Henry Farm, Oriole contains most of the fast food resturants, followed by Church and Wellesley and Steeles West, L'Amoreaux West Neighborhoods.

In [188]:
fastfood_places.groupby('Venue')['Neighborhood'].count().sort_values(ascending=False).to_frame().reset_index()

,Venue,Neighborhood
0,McDonald's,8
1,KFC,4
2,Crepe Delicious,4
3,iQ Food Co,3
4,The Burger's Priest,3
5,A&W,3
6,Wendy’s,2
7,New York Fries,1
8,New York Fried Chicken,1
9,Fit For Life,1


Most of the fast food resturents are McDonalds which is followed by KFC and Crepe Delicious.

Neighbourhoods without fast food resturensts. Potential locations.

In [189]:
temp_df = toronto_venues.copy()
temp_df["is_fastfood"] = (temp_df["Venue Category"] == 'Fast Food Restaurant')
temp_df = temp_df.groupby('Neighborhood')['is_fastfood'].sum().to_frame().reset_index()
temp_df.query("is_fastfood==0")

,Neighborhood,is_fastfood
0,Agincourt,0
1,"Alderwood, Long Branch",0
2,"Bathurst Manor, Wilson Heights, Downsview North",0
3,Bayview Village,0
5,Berczy Park,0
6,"Birch Cliff, Cliffside West",0
7,"Brockton, Parkdale Village, Exhibition Place",0
9,"CN Tower, King and Spadina, Railway Lands, Har...",0
10,Caledonia-Fairbanks,0
11,Canada Post Gateway Processing Centre,0


There are 72 Neighborhoods without a fast food resturents. So there is a real oppertunity here. 

## 1.3 Finding Similar Neighborhoods

Here we aims to cluster all the neighborhoods based on the the venue types they have. From this we can identify which neighborhoods are similar to the ones with fast food resturents. Then based on the most populous Neighborhoods we can suggest the best places to start the business. 

In [190]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# one hot encoding
torronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
torronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = list(torronto_onehot.columns)
# move the column to head of list using index, pop and insert
fixed_columns.insert(0, fixed_columns.pop(fixed_columns.index('Neighborhood')))
torronto_onehot = torronto_onehot[fixed_columns]

# Grouping by neighbohrhood
torronto_grouped = torronto_onehot.groupby('Neighborhood').mean().reset_index()

def get_population(neighbourhood):
    list_hoods = neighbourhood.lower().split(",")
    population = 0
    
    for hood in list_hoods:
        hood = hood.strip()
        population += dict_population.get(hood,0.0)
    return population

torronto_grouped['Population'] = torronto_grouped['Neighborhood'].apply(lambda x: get_population(x))

# top 10 venues per Neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torronto_grouped['Neighborhood']

for ind in np.arange(torronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted['Population'] = torronto_grouped['Population']

# clustering
kclusters = 30

torronto_grouped_clustering = torronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torronto_merged = neighborhood_data.rename(columns={'Neighbourhood': 'Neighborhood'})

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
torronto_merged = torronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# torronto_merged['Cluster Labels'] = torronto_merged['Cluster Labels'].astype('int')

torronto_merged=torronto_merged[~torronto_merged['Cluster Labels'].isna()]
torronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Population
0,M3A,North York,Parkwoods,43.753259,-79.329656,7.0,Population,Food & Drink Shop,Park,Construction & Landscaping,Fast Food Restaurant,Greek Restaurant,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,26533.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,18.0,Population,French Restaurant,Coffee Shop,Financial or Legal Service,Hockey Arena,Portuguese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,17047.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Restaurant,Sushi Restaurant,Bank,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,10.0,Population,Clothing Store,Miscellaneous Shop,Boutique,Carpet Store,Coffee Shop,Event Space,Furniture / Home Store,Gift Shop,Accessories Store,17519.0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Café,Diner,Persian Restaurant,Japanese Restaurant,Bar,Bank,Sushi Restaurant,Mexican Restaurant,Portuguese Restaurant,0.0


## 1.3 Examining Similar Neighborhoods

In [191]:
fast_neigh.merge(torronto_merged).head()

,Neighborhood,Venue,PostalCode,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Population
0,"Fairview, Henry Farm, Oriole",4,M2J,North York,43.778517,-79.346556,15.0,Population,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Japanese Restaurant,Juice Bar,Bank,Cosmetics Shop,Chocolate Shop,2790.0
1,Church and Wellesley,2,M4Y,Downtown Toronto,43.665860,-79.383160,4.0,Population,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Hotel,Pub,Café,Mediterranean Restaurant,13397.0
2,"Steeles West, L'Amoreaux West",2,M1W,Scarborough,43.799525,-79.318389,1.0,Fast Food Restaurant,Pharmacy,Chinese Restaurant,Sandwich Place,Bank,Supermarket,Breakfast Spot,Coffee Shop,Pizza Place,Electronics Store,0.0
3,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",2,M6M,York,43.691116,-79.476013,0.0,Population,Fast Food Restaurant,Discount Store,Fried Chicken Joint,Restaurant,Sandwich Place,Dumpling Restaurant,Drugstore,Donut Shop,Dessert Shop,21284.0
4,"Willowdale, Willowdale East",1,M2N,North York,43.770120,-79.408493,14.0,Population,Ramen Restaurant,Coffee Shop,Japanese Restaurant,Café,Pizza Place,Shopping Mall,Vietnamese Restaurant,Hotel,Electronics Store,43144.0


In [192]:
combined = fast_neigh.merge(torronto_merged).drop_duplicates(subset='Cluster Labels').reset_index(drop=True)
combined['rank'] = combined.index + 1
combined = combined[['rank','Cluster Labels']]
df1 = torronto_merged.groupby('Cluster Labels')['Neighborhood'].count().to_frame().reset_index().rename(columns={"Neighborhood":"count_all"})
df2 = fast_neigh.merge(torronto_merged).groupby('Cluster Labels')['Neighborhood'].count().to_frame().reset_index().rename(columns={"Neighborhood":"count_fastfood"})
df3 = df1.merge(df2).merge(combined)

df3['available_hoods'] = df3.count_all - df2.count_fastfood
df3.sort_values(by='rank')

,Cluster Labels,count_all,count_fastfood,rank,available_hoods
7,15.0,1,1,1,0
2,4.0,4,2,2,2
1,1.0,37,12,3,25
0,0.0,7,3,4,4
6,14.0,2,1,5,1
11,24.0,2,1,6,1
8,16.0,4,1,7,3
4,8.0,1,1,8,0
10,22.0,2,1,9,1
3,7.0,3,1,10,2


We can see that there are no neighbourhoods similar to the hood which have most number of fast food resturents. Therefore, we move to the second hood. We can see that there are 2 similar neighbourhoods which are similar.

In [193]:
cluster4 = torronto_merged[['PostalCode','Borough','Neighborhood','Latitude','Longitude','Population','Cluster Labels']].query("`Cluster Labels`==4.0")
cluster4 = cluster4.merge(temp_df)
cluster4

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Population,Cluster Labels,is_fastfood
0,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,13683.0,4.0,0
1,M4G,East York,Leaside,43.709060,-79.363452,13876.0,4.0,0
2,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,13749.0,4.0,1
3,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,13397.0,4.0,2


We can see that among 4 locations 2 locations can be used for the new Fast food resturant. As they have similar population either/both places are possible selections. 
- __Neighborhood__: West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale |     __Borough__:Etobicoke
- __Neighborhood__: Leaside	 |   __Borough__:East York




In [194]:
# Lets create markers for all the neighbourhoods in Toronto in a Folium map which is centered on Toronto
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.2)

neighborhood_data = cluster4.query("`is_fastfood`==0")

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_data['Latitude'], neighborhood_data['Longitude'], neighborhood_data['Borough'], neighborhood_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
for lat, lng, venue, neighborhood in zip(fastfood_places['Neighborhood Latitude'], fastfood_places['Neighborhood Longitude'], fastfood_places['Venue'], fastfood_places['Neighborhood']):
    label = '{}, {}'.format(neighborhood, venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='r',
        fill_opacity=0.3,
        opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The blue locations are the best loctions for the new fast food resturant.